In [ ]:
# Copyright 2023 Nils Knieling
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 💬 Chat with Vertex AI

[![Open in Colab](https://img.shields.io/badge/Open%20in%20Colab-%23F9AB00.svg?logo=googlecolab&logoColor=white)](https://colab.research.google.com/github/Cyclenerd/google-cloud-gcp-openai-api/blob/master/Vertex_AI_Chat.ipynb)
[![Open in Vertex AI Workbench](https://img.shields.io/badge/Open%20in%20Vertex%20AI%20Workbench-%234285F4.svg?logo=googlecloud&logoColor=white)](https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/Cyclenerd/google-cloud-gcp-openai-api/master/Vertex_AI_Chat.ipynb)
[![View on GitHub](https://img.shields.io/badge/View%20on%20GitHub-181717.svg?logo=github&logoColor=white)](https://github.com/Cyclenerd/google-cloud-gcp-openai-api/blob/master/Vertex_AI_Chat.ipynb)

This notebook describes how to deploy the [OpenAI API for Google Cloud Vertex AI](https://github.com/Cyclenerd/google-cloud-gcp-openai-api#readme) API backend and [Chatbot UI](https://github.com/mckaywrigley/chatbot-ui) frontend as Google Cloud Run service.

> By default, [**Google Cloud does not use Customer Data to train its foundation models**](https://cloud.google.com/vertex-ai/docs/generative-ai/data-governance#foundation_model_development) as part of Google Cloud's AI/ML Privacy Commitment. More details about how Google processes data can also be found in [Google's Customer Data Processing Addendum (CDPA)](https://cloud.google.com/terms/data-processing-addendum).

## Setup


### Configuration

Regions: <https://cloud.google.com/vertex-ai/docs/general/locations#vertex-ai-regions>

Model: One of the [foundation models](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#foundation_models) that are available in Vertex AI.

Key `openai_key` is used for authentication the chat frondend against the API backend application. It is not a real OpenAI API key.

In [ ]:
# @markdown ✏️ Replace the placeholder text below:

# Please fill in these values.
project_id = "your-google-cloud-project"  # @param {type:"string"}
region = "europe-west1"  # @param {type:"string"}
artifact_repository = "docker-openai-api"  # @param {type:"string"}
model = "codechat-bison"  # @param {type:"string"}
openai_key = "sk-XYZ-replace-with-good-key" # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"
assert region, "⚠️ Please provide a Google Cloud region"
assert artifact_repository, "⚠️ Please provide a Google Cloud Artifact repository name"
assert model, "⚠️ Please provide a valid model"
assert openai_key, "⚠️ Please provide a valid key for authentication"

# Configure gcloud.
!gcloud config set project "{project_id}"
!gcloud config set storage/parallel_composite_upload_enabled "True"

print(f"\n☁️ Google Cloud project ID: {project_id}")
print("☑️ Done")

### Authenticate

In [ ]:
#@markdown #### (Colab only!) Authenticate your Google Cloud Account

# Authenticate gcloud.
from google.colab import auth
auth.authenticate_user()

print("☑️ OK")

In [ ]:
#@markdown ####  Check authenticated user
current_user = !gcloud auth list \
  --filter="status:ACTIVE" \
  --format="value(account)" \
  --quiet

current_user = current_user[0]
print(f"Current user: {current_user}")

### APIs

In [ ]:
#@markdown #### Enable Google Cloud APIs
#@markdown > Only necessary if the APIs are not yet activated in the project!

# Enable APIs
my_google_apis = [
    "aiplatform.googleapis.com",
    "run.googleapis.com",
    "artifactregistry.googleapis.com",
    "cloudbuild.googleapis.com",
    "containeranalysis.googleapis.com",
    "containerscanning.googleapis.com",
]

for api in my_google_apis :
  print(f"Enable API: {api}")
  !gcloud services enable "{api}" \
    --project="{project_id}" \
    --quiet

print("☑️ OK")

### Registry

In [ ]:
#@markdown #### Create Artifact Registry repositoriy for Docker cointainer images
#@markdown > Only necessary if the repositoriy does not already exist in the project and region!

!gcloud artifacts repositories create "{artifact_repository}" \
  --repository-format="docker"\
  --description="Docker contrainer registry" \
  --location="{region}" \
  --project="{project_id}" \
  --quiet

print("☑️ Done")

## Deploy

### Backend

In [ ]:
#@markdown #### Build Docker cointainer for API backend

backend_git = "https://github.com/Cyclenerd/google-cloud-gcp-openai-api.git"  # @param {type:"string"}
backend_git_rev = "master"  # @param {type:"string"}
assert backend_git, "⚠️ Please provide a Git repo"
assert backend_git_rev, "⚠️ Please provide a Git revision"

!gcloud builds submit "{backend_git}" \
    --git-source-revision="{backend_git_rev}" \
    --tag="{region}-docker.pkg.dev/{project_id}/{artifact_repository}/vertex:latest" \
    --timeout="1h" \
    --region="{region}" \
    --default-buckets-behavior="regional-user-owned-bucket" \
    --quiet

In [ ]:
#@markdown #### Deploy Cloud Run service with API backend

!gcloud run deploy "openai-api-vertex" \
    --image="{region}-docker.pkg.dev/{project_id}/{artifact_repository}/vertex:latest" \
    --description="OpenAI API for Google Cloud Vertex AI" \
    --region="{region}" \
    --set-env-vars="OPENAI_API_KEY={openai_key},GOOGLE_CLOUD_LOCATION={region},MODEL_NAME={model}" \
    --max-instances=4 \
    --allow-unauthenticated \
    --quiet

### Frontend

* [Chatbot UI](https://github.com/mckaywrigley/chatbot-ui):
  * Git: `https://github.com/mckaywrigley/chatbot-ui.git`
  * Rev: `main`
* [Chatbot UI fork](https://github.com/Cyclenerd/chatbot-ui) with less OpenAI branding:
  * Git: `https://github.com/Cyclenerd/chatbot-ui.git`
  * Rev: `google`



In [ ]:
#@markdown #### Build Docker cointainer for chat frontend

frontend_git = "https://github.com/Cyclenerd/chatbot-ui.git"  # @param {type:"string"}
frontend_git_rev = "google"  # @param {type:"string"}

assert frontend_git, "⚠️ Please provide a Git repo"
assert frontend_git_rev, "⚠️ Please provide a Git revision"

!gcloud builds submit "{frontend_git}" \
    --git-source-revision="{frontend_git_rev}" \
    --tag="{region}-docker.pkg.dev/{project_id}/{artifact_repository}/chatbot-ui:main" \
    --timeout="1h" \
    --region="{region}" \
    --default-buckets-behavior="regional-user-owned-bucket" \
    --quiet

In [ ]:
#@markdown #### Deploy Cloud Run service with chat frontend

# @markdown ✏️ Replace the Cloud Run service URL from the backend below:

backend_url = "https://openai-api-vertex-XYZ-AB.a.run.app"  # @param {type:"string"}
assert backend_url, "⚠️ Please provide a Cloud Run backend URL"

!gcloud run deploy "chatbot-ui" \
    --image="{region}-docker.pkg.dev/{project_id}/{artifact_repository}/chatbot-ui:main" \
    --description="Chatbot UI" \
    --region="{region}" \
    --set-env-vars="OPENAI_API_KEY={openai_key},OPENAI_API_HOST={backend_url}" \
    --max-instances=2 \
    --allow-unauthenticated \
    --quiet

## 🗑️ Clean up

If you don't need the infrastructure anymore, you can delete it with the following snippets.

In [ ]:
#@markdown ### Delete Cloud Run service with chat frontend

!gcloud run services delete "chatbot-ui" \
  --region="{region}" \
  --project="{project_id}" \
  --quiet

In [ ]:
#@markdown ### Delete Cloud Run service with API backend

!gcloud run services delete "openai-api-vertex" \
  --region="{region}" \
  --project="{project_id}" \
  --quiet

In [ ]:
#@markdown ### Delete Artifact Registry repositoriy
!gcloud artifacts repositories delete "{artifact_repository}" \
  --location="{region}" \
  --project="{project_id}" \
  --quiet